In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\AlpOkul\TUe_Masters\Internship_BIASLab\CVI_AlpSemih\Project.toml`


In [2]:
using Random,LinearAlgebra, Flux.Optimise, Plots, ForneyLab
using ForneyLab:ParamStr2

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [3]:
Random.seed!(1234);

In [4]:
n = 5
m_data = 3.0
w_data = 4.0
y_data = sqrt(1/w_data)*randn(n) .+ m_data;

## VMP

In [5]:

g = FactorGraph()
# Priors
@RV m ~ GaussianMeanVariance(0.0, 100.0)
@RV w ~ Gamma(0.01, 0.01)

# Observarion model
y = Vector{Variable}(undef, n)
for i = 1:n
    @RV y[i] ~ GaussianMeanPrecision(m, w)
    placeholder(y[i], :y, index=i)
end

# Specify posterior factorization
q = PosteriorFactorization(m, w, ids=[:M, :W])
# Build the variational update algorithms for each posterior factor
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code for the algorithms
source_code = algorithmSourceCode(algo, free_energy=true)

# And inspect the algorithm code
#println(source_code)
eval(Meta.parse(source_code));
data = Dict(:y => y_data)

# Initial posterior factors
marginals = Dict(:m => vague(GaussianMeanVariance),
                 :w => vague(Gamma))

n_its = 2*n
F = Vector{Float64}(undef, n_its) # Initialize vector for storing Free energy
m_est = Vector{Float64}(undef, n_its)
w_est = Vector{Float64}(undef, n_its)
for i = 1:n_its
    stepM!(data, marginals)
    stepW!(data, marginals)
    println(mean(marginals[:m]))
end

2.943261165197841
2.9422682960987996
2.942070654327122
2.942031313297883
2.9420234824504488
2.942021923719979
2.9420216134547337
2.9420215516964507
2.942021539403469
2.9420215369565508


# Without Initial Step Size

### No Convergence Algo, Automatic Step Size, Constant Step Size

In [6]:
opt = ParamStr2(max_iterations=1e5,stepsize_update="none",convergence_algo="none",verbose=true)
# CVI Implementation
f(x) = x^6
# Priors
graph2 = FactorGraph()
@RV m ~ GaussianMeanVariance(1.0, 100.0)
@RV m_bar ~ Cvi(m,g=f,opt=opt,num_samples=1000,num_iterations=1)
@RV w ~ Gamma(0.01, 0.01)

# Observarion model
y = Vector{Variable}(undef, n)
for i = 1:n
    @RV y[i] ~ GaussianMeanPrecision(m_bar, w)
    placeholder(y[i], :y, index=i)
end

# Specify posterior factorization
q = PosteriorFactorization(m, w, ids=[:M, :W])
# Build the variational update algorithms for each posterior factor
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code for the algorithms
source_code = algorithmSourceCode(algo, free_energy=true)

# And inspect the algorithm code
#println(source_code)
eval(Meta.parse(source_code));


data = Dict(:y => y_data)

# Initial posterior factors
marginals2 = Dict(:m => vague(GaussianMeanVariance),
                 :w => vague(Gamma))

n_its = 2*n
F = Vector{Float64}(undef, n_its) # Initialize vector for storing Free energy
m_est = Vector{Float64}(undef, n_its)
w_est = Vector{Float64}(undef, n_its)
for i = 1:n_its
    stepM!(data, marginals2)
    stepW!(data, marginals2)
    println(mean(marginals2[:m]))
    println("---")
end



inexactLineSearch failed, setting initial stepsize to 1.0e-5
Importance ratios are 0, fitted Pareto shape parameter = NaN
1.0
---
Fitted Pareto shape parameter = -0.577467919589317
0.9999842167056614
---
Fitted Pareto shape parameter = -0.6158958032557569
1.005700102491064
---
Fitted Pareto shape parameter = -0.6052632984298429
1.000809722721084
---
Importance ratios are 0, fitted Pareto shape parameter = NaN
0.9999999999941968
---
Fitted Pareto shape parameter = -0.5470372202276892
1.0000007965060311
---
Importance ratios are 0, fitted Pareto shape parameter = NaN
1.0000000000167844
---
Fitted Pareto shape parameter = -0.6358846794982078
1.019523232234491
---
Fitted Pareto shape parameter = -0.513649776303838
1.0000000695424829
---
Fitted Pareto shape parameter = -0.5635962282250015
1.000000004512718
---


In [7]:
0.5*0.9^200

3.5275395543276837e-10

In [8]:
opt = ParamStr2(max_iterations=1e5,stepsize_update="adaptive",convergence_algo="none",verbose=true)
#opt = Descent(0.5)
# CVI Implementation
f(x) = x
# Priors
graph2 = FactorGraph()
@RV m ~ GaussianMeanVariance(1.0, 100.0)
@RV m_bar ~ Cvi(m,g=f,opt=opt,num_samples=1000,num_iterations=1000)
@RV w ~ Gamma(0.01, 0.01)

# Observarion model
y = Vector{Variable}(undef, n)
for i = 1:n
    @RV y[i] ~ GaussianMeanPrecision(m_bar, w)
    placeholder(y[i], :y, index=i)
end

# Specify posterior factorization
q = PosteriorFactorization(m, w, ids=[:M, :W])
# Build the variational update algorithms for each posterior factor
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code for the algorithms
source_code = algorithmSourceCode(algo, free_energy=true)

# And inspect the algorithm code
#println(source_code)
eval(Meta.parse(source_code));


data = Dict(:y => y_data)

# Initial posterior factors
marginals2 = Dict(:m => vague(GaussianMeanVariance),
                 :w => vague(Gamma))

n_its = 2*n
F = Vector{Float64}(undef, n_its) # Initialize vector for storing Free energy
m_est = Vector{Float64}(undef, n_its)
w_est = Vector{Float64}(undef, n_its)
for i = 1:n_its
    stepM!(data, marginals2)
    stepW!(data, marginals2)
    println(mean(marginals2[:m]))
    println("---")
end



Importance ratios are 0, fitted Pareto shape parameter = NaN
2.814583582072744
---
Warning, fitted Pareto shape parameter =0.8306631844210011>=0.7!
2.929148578340885
---
Warning, fitted Pareto shape parameter =1.0985896677343712>=0.7!
2.9408421844533637
---
Fitted Pareto shape parameter = 0.4764553158658591
2.8885776936645984
---
Warning, fitted Pareto shape parameter =1.0351646408116881>=0.7!
2.9402200901841895
---
Warning, fitted Pareto shape parameter =0.8629416597643405>=0.7!
2.9321968475636013
---
Fitted Pareto shape parameter = 0.168020232907223
2.88799961401796
---
Warning, fitted Pareto shape parameter =0.9675459246038264>=0.7!
2.936672125396336
---
Warning, fitted Pareto shape parameter =1.0870097113916484>=0.7!
2.9358386334435465
---
Fitted Pareto shape parameter = 0.5545350759086243
2.9009245310438727
---


In [13]:
opt = ParamStr2(max_iterations=1e5,stepsize_update="none",convergence_algo="none",verbose=true)
#opt = Descent(0.5)
# CVI Implementation
f(x) = x
# Priors
graph2 = FactorGraph()
@RV m ~ GaussianMeanVariance(1.0, 100.0)
@RV m_bar ~ Cvi(m,g=f,opt=opt,num_samples=1000,num_iterations=1000)
@RV w ~ Gamma(0.01, 0.01)

# Observarion model
y = Vector{Variable}(undef, n)
for i = 1:n
    @RV y[i] ~ GaussianMeanPrecision(m_bar, w)
    placeholder(y[i], :y, index=i)
end

# Specify posterior factorization
q = PosteriorFactorization(m, w, ids=[:M, :W])
# Build the variational update algorithms for each posterior factor
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code for the algorithms
source_code = algorithmSourceCode(algo, free_energy=true)

# And inspect the algorithm code
#println(source_code)
eval(Meta.parse(source_code));


data = Dict(:y => y_data)

# Initial posterior factors
marginals2 = Dict(:m => vague(GaussianMeanVariance),
                 :w => vague(Gamma))

n_its = 2*n
F = Vector{Float64}(undef, n_its) # Initialize vector for storing Free energy
m_est = Vector{Float64}(undef, n_its)
w_est = Vector{Float64}(undef, n_its)
for i = 1:n_its
    stepM!(data, marginals2)
    stepW!(data, marginals2)
    println(mean(marginals2[:m]))
    println("---")
end



inexactLineSearch succeeded, setting initial stepsize to 5.886836577006101e-8
Importance ratios are 0, fitted Pareto shape parameter = NaN
1.0131540556020504
---
Warning, fitted Pareto shape parameter =1.0431518225868692>=0.7!
1.8334666777010793
---
Warning, fitted Pareto shape parameter =1.0905691266185282>=0.7!
1.090252732988495
---
Warning, fitted Pareto shape parameter =1.093801073824839>=0.7!
1.0552114211790404
---
Warning, fitted Pareto shape parameter =1.0936010980313406>=0.7!
1.05875707992083
---
Warning, fitted Pareto shape parameter =1.095107629398534>=0.7!
1.0499259471418694
---
Warning, fitted Pareto shape parameter =1.0942829138817742>=0.7!
1.0601548313565239
---
Warning, fitted Pareto shape parameter =1.0942603830257882>=0.7!
1.0539831013361367
---
Warning, fitted Pareto shape parameter =1.0948941739648068>=0.7!
1.0569732009423511
---
Warning, fitted Pareto shape parameter =1.0949016081520524>=0.7!
1.0565421406196136
---


In [14]:
opt.ḡ

In [15]:
opt.h̄

+ (generic function with 308 methods)